In [10]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

In [23]:
#  Import and read the charity_data.csv.
url = "https://raw.githubusercontent.com/kevogil/charity-funding-predictor/main/Resources/charity_data.csv"
application_df = pd.read_csv(url)

# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=['EIN', 'NAME', 'ORGANIZATION'], inplace=True)

# Look at APPLICATION_TYPE value counts for binning
app_cnt = application_df['APPLICATION_TYPE'].value_counts()

# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff_point = 500
application_types_to_replace = app_cnt[app_cnt < cutoff_point].index.tolist()


# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Look at CLASSIFICATION value counts for binning
class_cnt = application_df['CLASSIFICATION'].value_counts()

# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff_point = 1000
classifications_to_replace = class_cnt[class_cnt < cutoff_point].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Convert categorical data to numeric with `pd.get_dummies`
numeric_application_df = pd.get_dummies(application_df)

# Split our preprocessed data into our features and target arrays
y = numeric_application_df['IS_SUCCESSFUL']
x = numeric_application_df.drop(columns=['IS_SUCCESSFUL'])

# Split the preprocessed data into a training and testing dataset
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
x_scaler = scaler.fit(x_train)

# Scale the data
x_train_scaled = x_scaler.transform(x_train)
x_test_scaled = x_scaler.transform(x_test)

In [24]:
### Compile, Train and Evaluate the Model

In [25]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(x_train_scaled[0])
hidden_nodes_layer1 =  16
hidden_nodes_layer2 = 10
hidden_nodes_layer3 = 8

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 16)                640       
                                                                 
 dense_10 (Dense)            (None, 10)                170       
                                                                 
 dense_11 (Dense)            (None, 8)                 88        
                                                                 
 dense_12 (Dense)            (None, 1)                 9         
                                                                 
Total params: 907
Trainable params: 907
Non-trainable params: 0
_________________________________________________________________


In [26]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [27]:
# Train the model
fit_model = nn.fit(x_train_scaled,y_train,epochs=200)

Epoch 1/200
804/804 [==============================] - 2s 2ms/step - loss: 0.6059 - accuracy: 0.6949
Epoch 2/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5653 - accuracy: 0.7234
Epoch 3/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5613 - accuracy: 0.7262
Epoch 4/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5603 - accuracy: 0.7264
Epoch 5/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5593 - accuracy: 0.7271
Epoch 6/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5584 - accuracy: 0.7273
Epoch 7/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5582 - accuracy: 0.7272
Epoch 8/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5579 - accuracy: 0.7277
Epoch 9/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5574 - accuracy: 0.7282
Epoch 10/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5568 - accura

In [28]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(x_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5690 - accuracy: 0.7251 - 409ms/epoch - 2ms/step
Loss: 0.569019079208374, Accuracy: 0.7251312136650085


In [29]:
# Save and export your results to an HDF5 file
from google.colab import files

nn.save('AlphabetSoupCharity_Optimization_v1.h5')
files.download('AlphabetSoupCharity_Optimization_v1.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>